# --- Day 4: Repose Record ---

You've sneaked into another supply closet - this time, it's across from the prototype suit manufacturing lab. You need to sneak inside and fix the issues with the suit, but there's a guard stationed outside the lab, so this is as close as you can safely get.

As you search the closet for anything that might help, you discover that you're not the first person to want to sneak in. Covering the walls, someone has spent an hour starting every midnight for the past few months secretly observing this guard post! They've been writing down the ID of the one guard on duty that night - the Elves seem to have decided that one guard was enough for the overnight shift - as well as when they fall asleep or wake up while at their post (your puzzle input).

For example, consider the following records, which have already been organized into chronological order:

```
[1518-11-01 00:00] Guard #10 begins shift
[1518-11-01 00:05] falls asleep
[1518-11-01 00:25] wakes up
[1518-11-01 00:30] falls asleep
[1518-11-01 00:55] wakes up
[1518-11-01 23:58] Guard #99 begins shift
[1518-11-02 00:40] falls asleep
[1518-11-02 00:50] wakes up
[1518-11-03 00:05] Guard #10 begins shift
[1518-11-03 00:24] falls asleep
[1518-11-03 00:29] wakes up
[1518-11-04 00:02] Guard #99 begins shift
[1518-11-04 00:36] falls asleep
[1518-11-04 00:46] wakes up
[1518-11-05 00:03] Guard #99 begins shift
[1518-11-05 00:45] falls asleep
[1518-11-05 00:55] wakes up
```

Timestamps are written using `year-month-day hour:minute` format. The guard falling asleep or waking up is always the one whose shift most recently started. Because all asleep/awake times are during the midnight hour (00:00 - 00:59), only the minute portion (00 - 59) is relevant for those events.

Visually, these records show that the guards are asleep at these times:

```
Date   ID   Minute
            000000000011111111112222222222333333333344444444445555555555
            012345678901234567890123456789012345678901234567890123456789
11-01  #10  .....####################.....#########################.....
11-02  #99  ........................................##########..........
11-03  #10  ........................#####...............................
11-04  #99  ....................................##########..............
11-05  #99  .............................................##########.....
```

The columns are Date, which shows the month-day portion of the relevant day; ID, which shows the guard on duty that day; and Minute, which shows the minutes during which the guard was asleep within the midnight hour. (The Minute column's header shows the minute's ten's digit in the first row and the one's digit in the second row.) Awake is shown as `.`, and asleep is shown as `#`.

Note that guards count as asleep on the minute they fall asleep, and they count as awake on the minute they wake up. For example, because `Guard #10` wakes up at `00:25` on `1518-11-01`, minute 25 is marked as awake.

If you can figure out the guard most likely to be asleep at a specific time, you might be able to trick that guard into working tonight so you can have the best chance of sneaking in. You have two strategies for choosing the best guard/minute combination.

**Strategy 1:** Find the guard that has the most minutes asleep. What minute does that guard spend asleep the most?

In the example above, `Guard #10` spent the most minutes asleep, a total of 50 minutes `(20+25+5)`, while `Guard #99` only slept for a total of 30 minutes `(10+10+10)`. `Guard #10` was asleep most during minute 24 (on two days, whereas any other minute the guard was asleep was only seen on one day).

While this example listed the entries in chronological order, your entries are in the order you found them. You'll need to organize them before they can be analyzed.

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be `10 * 24 = 240`.)

In [23]:
import datetime as dt
import itertools as it
import re

from collections import Counter, defaultdict

In [1]:
def test_input1():
    yield '[1518-11-01 00:00] Guard #10 begins shift'
    yield '[1518-11-01 00:05] falls asleep'
    yield '[1518-11-01 00:25] wakes up'
    yield '[1518-11-01 00:30] falls asleep'
    yield '[1518-11-01 00:55] wakes up'
    yield '[1518-11-01 23:58] Guard #99 begins shift'
    yield '[1518-11-02 00:40] falls asleep'
    yield '[1518-11-02 00:50] wakes up'
    yield '[1518-11-03 00:05] Guard #10 begins shift'
    yield '[1518-11-03 00:24] falls asleep'
    yield '[1518-11-03 00:29] wakes up'
    yield '[1518-11-04 00:02] Guard #99 begins shift'
    yield '[1518-11-04 00:36] falls asleep'
    yield '[1518-11-04 00:46] wakes up'
    yield '[1518-11-05 00:03] Guard #99 begins shift'
    yield '[1518-11-05 00:45] falls asleep'
    yield '[1518-11-05 00:55] wakes up'

In [2]:
def input_():
    with open('./inputs/day04/input.txt') as fp:
        yield from sorted(line.strip() for line in fp)

In [12]:
DT_FMT = '%Y-%m-%d %H:%M'

In [8]:
GUARD_RE = re.compile(r'^\[(?P<timestamp>.{16})\] Guard #(?P<id_>\d+) begins shift$')

In [9]:
GUARD_RE.match('[1518-11-01 00:00] Guard #10 begins shift')

<re.Match object; span=(0, 41), match='[1518-11-01 00:00] Guard #10 begins shift'>

In [15]:
Out[9]['timestamp'], Out[9]['id_']

('1518-11-01 00:00', '10')

In [14]:
dt.datetime.strptime(Out[9]['timestamp'], DT_FMT)

datetime.datetime(1518, 11, 1, 0, 0)

In [17]:
def guard_sleeps(events):
    sleeps = defaultdict(list)
    guard_id = None
    sleep_start = None
    
    for event in events:
        match = GUARD_RE.match(event)
        
        if match:
            guard_id = int(match['id_'])
        elif event.endswith('falls asleep'):
            sleep_start = int(event[15:17])
        elif event.endswith('wakes up'):
            sleep_end = int(event[15:17])
            sleeps[guard_id].append(list(range(sleep_start, sleep_end)))
    
    return sleeps

In [19]:
guard_sleeps(test_input1())

defaultdict(list,
            {10: [[5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24],
              [30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,
               46,
               47,
               48,
               49,
               50,
               51,
               52,
               53,
               54],
              [24, 25, 26, 27, 28]],
             99: [[40, 41, 42, 43, 44, 45, 46, 47, 48, 49],
              [36, 37, 38, 39

In [20]:
def sum_sleeps(sleep_lists):
    return sum(l[-1] - l[0] + 1 for l in sleep_lists)


def sleepiest_guard(sleeps):
    return max((guard for guard in sleeps), key=lambda g: sum_sleeps(sleeps[g]))

In [21]:
sleepiest_guard(guard_sleeps(test_input1()))

10

In [30]:
def sleepiest_minute(sleeps):
    guard = sleepiest_guard(sleeps)
    minutes_asleep = sleeps[guard]
    counts = Counter(it.chain.from_iterable(minutes_asleep))
    return guard, counts.most_common(1)[0][0]

In [31]:
sleepiest_minute(guard_sleeps(test_input1()))

(10, 24)

In [32]:
sleepiest_minute(guard_sleeps(input_()))

(727, 29)

In [33]:
727 * 29

21083

# --- Part Two ---

**Strategy 2:** Of all guards, which guard is most frequently asleep on the same minute?

In the example above, `Guard #99` spent minute 45 asleep more than any other guard or minute - three times in total. (In all other cases, any guard spent any minute asleep at most twice.)

What is the ID of the guard you chose multiplied by the minute you chose? (In the above example, the answer would be `99 * 45 = 4455`.)

In [34]:
def sleepiest_minute2(sleeps):
    return {
        guard: Counter(it.chain.from_iterable(sleep_lists)).most_common(1) 
        for guard, sleep_lists in sleeps.items()
    }

In [38]:
max(sleepiest_minute2(guard_sleeps(test_input1())).items(), key=lambda i: i[1][0][1])

(99, [(45, 3)])

In [39]:
max(sleepiest_minute2(guard_sleeps(input_())).items(), key=lambda i: i[1][0][1])

(1657, [(32, 17)])

In [40]:
1657 * 32

53024